In [26]:
# Importing Libraries 
import pandas as pd
import psycopg2
import awswrangler as wr
import extract_data

dir(extract_data)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__warningregistry__',
 'extract_csv',
 'extract_json',
 'extract_sql',
 'merge_df',
 'pd',
 'psycopg2']

EXTRACTION

In [27]:

# File Paths
csv_file_path = r'C:\Users\Kartikay\Downloads\MOCK_DATA_CSV.csv'
json_path = r'C:\Users\Kartikay\Downloads\mock_data_json.json'

# Calling the function in extract_data.py to extract data from different sources
df_sql = extract_data.extract_sql(
    dbname='data_silos',
    user ='postgres',
    password ='postgres',
    host='localhost'
    )
df_csv = extract_data.extract_csv(csv_file_path)
df_json =  extract_data.extract_json(json_path)

# Consoliudating all the data sources into one common master dataframe 
df_master = extract_data.merge_df([df_sql, df_csv, df_json])

c:\Users\Kartikay\Desktop\Data Engineering\Data-Engineering\DataSilos\extract_data.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query,con=conn1)


 TRANSFORMATION

In [28]:
df_master.info()

df_master = df_master.drop(["product_id"], axis = 1)

df_master.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           3000 non-null   int64  
 1   product_name         3000 non-null   object 
 2   product_category     3000 non-null   object 
 3   product_price        2798 non-null   float64
 4   product_description  1971 non-null   object 
 5   manufacturer         2770 non-null   object 
 6   stock_quantity       2252 non-null   float64
 7   weight               1534 non-null   float64
 8   color                2086 non-null   object 
 9   release_date         2462 non-null   object 
 10  expiry_date          2389 non-null   object 
 11  master_id            3000 non-null   int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 281.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column           

In [29]:
df_master['product_category'].unique()
df_master[['product_category','product_sub_category']] = df_master['product_category'].str.split('-',n=1,expand = True) 


In [30]:
# DROP release and expiry columns as data is skewed

df_master.drop(['release_date', 'expiry_date', 'weight'], axis= 1, inplace = True)

In [31]:
df_master['product_category'] = df_master['product_category'].str.strip()
df_master['product_category'].unique()

array(['Food', 'Clothing', 'Outdoor', 'Health', 'Photography', 'Pets',
       'Kitchen', 'Home', 'Books', 'Storage', 'Office', 'Music',
       'Accessories', 'Fitness', 'Automotive', 'Toys', 'Sports', 'Garden',
       'Travel', 'Smart Home', 'Electronics', 'Audio', 'Gaming',
       'Home Improvement', 'Beauty', 'Tools', 'Home Appliances',
       'Furniture', 'Home Security', 'Footwear', 'Art Supplies', 'Crafts',
       'Wearable Tech', 'Bicycles', 'Apps', 'Baby', 'Safety', 'Bathroom',
       'Grooming'], dtype=object)

In [32]:
# Filling the NA Values according to the most correct format prescribed
df_master.fillna({
    "product_sub_category": "unknown",
    "product_description" : "unknown",
    "product_price" : df_master['product_price'].median()
}, inplace = True)

In [33]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          3000 non-null   object 
 1   product_category      3000 non-null   object 
 2   product_price         3000 non-null   float64
 3   product_description   3000 non-null   object 
 4   manufacturer          2770 non-null   object 
 5   stock_quantity        2252 non-null   float64
 6   color                 2086 non-null   object 
 7   master_id             3000 non-null   int64  
 8   product_sub_category  3000 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 211.1+ KB


In [34]:
df_master['stock_quantity'].max()

1000.0

VALIDATION

In [35]:
import pandera.pandas as pa
schema = pa.DataFrameSchema(
    {
        "product_name" : pa.Column(pa.String),
        "product_price" : pa.Column(float,pa.Check.in_range(0,1000)),
        "stock_quantity" : pa.Column(float,pa.Check.in_range(0,1000), nullable = True),
        "master_id": pa.Column(pa.Int64,pa.Check.less_than_or_equal_to(3000))
    }
)

validated_scheme = schema.validate(df_master)

LOAD

In [36]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          3000 non-null   object 
 1   product_category      3000 non-null   object 
 2   product_price         3000 non-null   float64
 3   product_description   3000 non-null   object 
 4   manufacturer          2770 non-null   object 
 5   stock_quantity        2252 non-null   float64
 6   color                 2086 non-null   object 
 7   master_id             3000 non-null   int64  
 8   product_sub_category  3000 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 211.1+ KB


In [37]:
# Upload the validated dataframe to S3 as a parquet file using awswrangler
wr.s3.to_parquet(df_master,path="s3://datasilosproj/folder/datasilosproj.parquet",index=False)

{'paths': ['s3://datasilosproj/folder/datasilosproj.parquet'],
 'partitions_values': {}}